# 1.0 An end-to-end classification problem (ETL)



## 1.1 Dataset description

The notebooks focus on a borrower's **credit modeling problem**. The database was downloaded through a dataquest project and is available at link below. The data is from **Lending Club** and contains data from loans made in the period **2007 to 2011**. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. The **target variable**, or what we are wanting to predict, is whether or not, given a person's history, they will repay the loan.

You can download the data from the [Kaggle](https://www.kaggle.com/datasets/samaxtech/lending-club-20072011-data).

Let's take the following steps:

1. Load Libraries
2. Fetch Data, including EDA
3. Pre-procesing
4. Data Segregation

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>

## 1.2 Install and load libraries

In [1]:
# !pip install wandb

In [2]:
import wandb
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 1.3 Preprocessing

### 1.3.1 Login wandb


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

WANDB_API_KEY=os.environ.get('WANDB_API_KEY')

In [6]:
# Login to Weights & Biases
!wandb login --relogin $WANDB_API_KEY

wandb: Appending key for api.wandb.ai to your netrc file: /Users/phamdinhkhanh/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


### 1.3.2 Artifacts

In [7]:
input_artifact="risk_credit/raw_data.csv:latest"
artifact_name="preprocessed_data.csv"
artifact_type="clean_data"
artifact_description="Data after preprocessing"

### 1.3.3 Setup your wandb project and clean the dataset

After the fetch step the raw data artifact was generated.
Now, we need to pre-processing the raw data to create a new artfiact (clean_data).

In [8]:
# Create a new job_type
run = wandb.init(project="risk_credit", job_type="process_data")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aikhanhblog (aikhanhblog-datascienceworld-kan). Use `wandb login --relogin` to force relogin


In [9]:
# Donwload the latest version of artifact raw_data.csv
artifact = run.use_artifact(input_artifact)

# Create a dataframe from the artifact
df = pd.read_csv(artifact.file())

In [10]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [11]:
print("Shape before cleaning: ", df.shape)

Shape before cleaning:  (42538, 52)


The target column (loan status) has values like Fully Paid, Charged Off, Late, In Grace Period etc., we will select the rows with the values Fully Paid and Charged Off.

In [12]:
# Selecting rows with the Fully Paid and Charged Off
df = df[((df['loan_status'] == 'Fully Paid') | (df['loan_status'] == 'Charged Off'))]

In [13]:
df['loan_status']

0         Fully Paid
1        Charged Off
2         Fully Paid
3         Fully Paid
5         Fully Paid
            ...     
39781     Fully Paid
39782     Fully Paid
39783     Fully Paid
39784     Fully Paid
39785     Fully Paid
Name: loan_status, Length: 38770, dtype: object

In [14]:
df['loan_status'].value_counts()

loan_status
Fully Paid     33136
Charged Off     5634
Name: count, dtype: int64

In [15]:
# Dropping irrelevant and redundant columns
columns_to_drop_1 = ['id','member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d',
                   'zip_code']
df.drop(columns_to_drop_1, axis=1, inplace=True)

In [16]:
# Dropping columns with data leakage
columns_to_drop_2 = ['out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp',
                   'total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee',
                   'last_pymnt_d', 'last_pymnt_amnt']
df.drop(columns_to_drop_2, axis=1, inplace=True)

In [17]:
# Dropping columns with only one unique value
after_columns = df.columns
drop_columns = []

for c in after_columns:
  # Drop any null values
  col_series = df[c].dropna().unique()

  # Remove the columns if there is only one unique value
  if(len(col_series) == 1):
    drop_columns.append(c)

df.drop(drop_columns, axis=1, inplace=True)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [18]:
print("Shape after dropping cols: ", df.shape)

Shape after dropping cols:  (38770, 23)


In [19]:
#Verifing null lines
null_counts = df.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

In [20]:
#Deleting pub_rec_bankruptcies
df.drop(['pub_rec_bankruptcies'], axis=1, inplace=True)
df = df.dropna()

In [21]:
print("Shape after cleaning: ", df.shape)

Shape after cleaning:  (37675, 22)


In [22]:
print(df.dtypes.value_counts())

object     12
float64    10
Name: count, dtype: int64


In [23]:
df['loan_status'].value_counts()

loan_status
Fully Paid     32286
Charged Off     5389
Name: count, dtype: int64

In [24]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(df[c].value_counts())

home_ownership
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: count, dtype: int64
verification_status
Not Verified       16281
Verified           11856
Source Verified     9538
Name: count, dtype: int64
emp_length
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: count, dtype: int64
term
 36 months    28234
 60 months     9441
Name: count, dtype: int64
addr_state
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY

In [25]:
print(df['title'].value_counts())
print(df['purpose'].value_counts())

title
Debt Consolidation            2068
Debt Consolidation Loan       1599
Personal Loan                  624
Consolidation                  488
debt consolidation             466
                              ... 
Legal                            1
Ryan's Debt Consolidation        1
Be Debt Free                     1
Discover HSBC                    1
JAL Loan                         1
Name: count, Length: 18881, dtype: int64
purpose
debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: count, dtype: int64


In [26]:
# For performance questions, let's drop the following columns
df.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1, inplace=True)

In [27]:
# let's convert 'int_rate' and 'revol_util' to float columns
df["int_rate"] = df["int_rate"].str.rstrip("%").astype("float")
df["revol_util"] = df["revol_util"].str.rstrip("%").astype("float")

In [28]:
df.shape

(37675, 18)

In [29]:
df.dtypes.value_counts()

float64    12
object      6
Name: count, dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37675 entries, 0 to 39785
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            37675 non-null  float64
 1   term                 37675 non-null  object 
 2   int_rate             37675 non-null  float64
 3   installment          37675 non-null  float64
 4   emp_length           37675 non-null  object 
 5   home_ownership       37675 non-null  object 
 6   annual_inc           37675 non-null  float64
 7   verification_status  37675 non-null  object 
 8   loan_status          37675 non-null  object 
 9   purpose              37675 non-null  object 
 10  dti                  37675 non-null  float64
 11  delinq_2yrs          37675 non-null  float64
 12  inq_last_6mths       37675 non-null  float64
 13  open_acc             37675 non-null  float64
 14  pub_rec              37675 non-null  float64
 15  revol_bal            37675 non-null  floa

In [31]:
df.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
5,5000.0,36 months,7.90,156.46,3 years,RENT,36000.0,Source Verified,Fully Paid,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


In [32]:
df.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
count,37675.000000,37675.000000,37675.00000,3.767500e+04,37675.000000,37675.000000,37675.000000,37675.000000,37675.000000,37675.000000,37675.000000,37675.000000
mean,11150.426012,11.974345,325.05027,6.946096e+04,13.292569,0.147074,0.872037,9.307764,0.053749,13389.439735,48.871170,22.106038
std,7367.968546,3.688724,208.87092,6.462082e+04,6.661638,0.493200,1.071154,4.395002,0.234639,15893.804160,28.336961,11.408414
min,500.000000,5.420000,15.69000,4.000000e+03,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000
25%,5500.000000,8.975000,167.56500,4.100000e+04,8.170000,0.000000,0.000000,6.000000,0.000000,3712.000000,25.500000,14.000000
50%,10000.000000,11.830000,280.62000,6.000000e+04,13.390000,0.000000,1.000000,9.000000,0.000000,8868.000000,49.300000,20.000000
75%,15000.000000,14.420000,430.05500,8.300000e+04,18.560000,0.000000,1.000000,12.000000,0.000000,17043.000000,72.400000,29.000000
max,35000.000000,24.590000,1305.19000,6.000000e+06,29.990000,11.000000,8.000000,44.000000,4.000000,149588.000000,99.900000,90.000000


In [33]:
# Delete duplicated rows
df.drop_duplicates(inplace=True)

# Generate a "clean data file"
df.to_csv(artifact_name,index=False)

In [34]:
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name=artifact_name,
                          type=artifact_type,
                          description=artifact_description)
artifact.add_file(artifact_name)

ArtifactManifestEntry(path='preprocessed_data.csv', digest='/nEo6o4VzA5+PJDHkPUQnQ==', size=4963598, local_path='/Users/phamdinhkhanh/Library/Application Support/wandb/artifacts/staging/tmp3j9ymrm0', skip_cache=False)

In [35]:
# Upload the artifact to Wandb
run.log_artifact(artifact)

<Artifact preprocessed_data.csv>

In [36]:
# close the run
# waiting a while after run the previous cell before execute this
run.finish()